# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [89]:
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
import numpy as np
import joblib, pickle

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
subscription_id = '8bb47da5-84b5-43cf-bd4a-97928e5c9b08'
resource_group = 'aml-quickstarts-127164'
workspace_name = 'quick-starts-ws-127164'

ws = Workspace.get(name="quick-starts-ws-127164")
exp = Experiment(workspace=ws, name="Ranga")

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='mpg')
ds = dataset.to_pandas_dataframe()

ds = ds.replace('?',0)
ds.dropna(inplace=True)

ds.drop(columns=['car name'],inplace=True)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [3]:
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=20,
    task='regression',
    primary_metric='r2_score',
    training_data=ds,
    label_column_name='mpg',
    n_cross_validations=3)

In [4]:
# TODO: Submit your experiment
autoexp = Experiment(workspace=ws,name='AutoRanga')
remote_run = autoexp.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       DONE
DESCRIPTION:  High cardinality features were detected in your inputs and handled.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization
DETAILS:      High cardinality features refer to columns that contain a large percentage of unique values.
+---------------------------------+---------------------------------+
|Column name                      |Column Content Type              |
+=============================

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:00:26       0.7822    0.7822
         1   MaxAbsScaler XGBoostRegressor                  0:00:21       0.8192    0.8192
         2   StandardScalerWrapper XGBoostRegressor         0:00:28       0.8308    0.8308
         3   StandardScalerWrapper RandomForest             0:00:29      

{'runId': 'AutoML_be220c23-6c40-476b-a2b6-87b7339be461',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-11-21T14:35:52.821638Z',
 'endTimeUtc': '2020-11-21T14:57:15.527856Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoRanga","subscription_id":"8bb47da5-84b5-43cf-bd4a-97928e5c9b08","resource_group":"aml-quickstarts-127164","workspace_name":"quick-starts-ws-127164","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"r2_score","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [60]:
best,fitted = remote_run.get_output()
print(best)

Run(Experiment: AutoRanga,
Id: AutoML_be220c23-6c40-476b-a2b6-87b7339be461_52,
Type: None,
Status: Completed)


In [61]:
print(fitted)

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

In [78]:
#TODO: Save the best model
filename = 'best_automl_model.sav'
# pickle.dump(fitted, open(filename, 'wb'))

joblib.dump(fitted,filename)

['best_automl_model.sav']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [91]:
registeredfinal = Model.register(workspace=ws,model_path='./training/best_automl_model.sav',model_name='registered.sav')

Registering model registered.sav


In [108]:
Model.get_model_path('registered.sav')

'azureml-models/registered.sav/4/best_automl_model.sav'

In [115]:
#Create inference config
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(workspace=ws,name='AzureML-AutoML')

inferencefinal = InferenceConfig(entry_script='score_11_18.py',environment=env) 

#Deploy model and check service state
from azureml.core.webservice import LocalWebservice


deployfinal = LocalWebservice.deploy_configuration(port=9050)
svcs = Model.deploy(workspace=ws,name='deployatlast',models=[registeredfinal],inference_config=inferencefinal,deployment_config=deployfinal)
svcs.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_9cc1a26fb8a5f7bac5f3a2cd8086e190
 ---> 614b72a6c7ae
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 021110e12a34
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjhiYjQ3ZGE1LTg0YjUtNDNjZi1iZDRhLTk3OTI4ZTVjOWIwOCIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEyNzE2NCIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEyNzE2NCIsIndvcmtzcGFjZUlkIjoiMjgyNjE0ZmUtNzU1MS00ZTMzLThkODEtYWE4YjMwMzE5MjQ1In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 4b8fc9f63355
 ---> 4deb2b5bf1dd
Step 4/5 : RUN mv '/var/azureml-app/tmprpxizj9x.py' /var/azureml-app/main.py
 ---> Running in ffc6b1962820
 ---> bcd4ea57db1d
Step 5/5 : 

In [117]:
print(svcs.state)

running


TODO: In the cell below, send a request to the web service you deployed to test it.

In [126]:
print(svcs.scoring_uri)

TypeError: 'NoneType' object is not subscriptable

In [20]:
import requests
import json

# URL for the web service
scoring_uri = '<your web service URI>'

data = np.array([4, 122, 87, 2332, 12, 70, 2])
data = data.reshape(1,-1)

# Set the content type
headers = {'Content-Type': 'application/json'}


# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print("prediction:",resp.text)

NameError: name 'key' is not defined

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(svcs.get_logs())

In [ ]:
svcs.delete()

### Convert model to ONNX format

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(fitted, initial_types=initial_type)
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [ ]:
comp_cluster.delete()